In [1]:
from z3 import *
import numpy as np
from itertools import combinations
from typing import Sequence
from tqdm.notebook import tqdm

Read instance file:

In [2]:
input_filename = '../../Instances/12x12.txt'

w, h, n, DX, DY = None, None, None, None, None

with open(input_filename, 'r') as f_in:
    lines = f_in.read().splitlines()

    split = lines[0].split(' ')
    w = int(split[0])
    h = int(split[1])

    n = int(lines[1])

    DX = []
    DY = []

    for i in range(int(n)):
        split = lines[i+2].split(' ')
        DX.append(int(split[0]))
        DY.append(int(split[1]))

Solver:

In [3]:
solver = Solver()

Model:

In [4]:
B = [[[Bool(f'B_{i}_{j}_{k}') for k in range(n)] for j in range(w)] for i in range(h)]
R = [Bool(f'R_{k}') for k in range(n)]

In [5]:
def at_least_one(bool_vars: Sequence):
    return Or(bool_vars)

def at_most_one(bool_vars: Sequence):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

In [6]:
# Constraint "at most one piece"
for i in range(h):
    for j in range(w):
        solver.add(at_most_one(B[i][j]))

In [7]:
# Iterate over all the pieces p
for p in tqdm(range(n), leave=False):    
    dx = DX[p]
    dy = DY[p]
    
    package_clauses = []
    
    # Iterate over all the coordinates where p can fit
    for i in range(h - dy + 1):
        for j in range(w - dx + 1):
            
            patch_clauses = []
            # Iterate over the cells of p's patch
            for f1 in range(dy):
                for f2 in range(dx):
                    patch_clauses.append(B[i + f1][j + f2][p])
            
            package_clauses.append(And(patch_clauses))
            
    solver.add(Or(R[p], at_least_one(package_clauses)))
                
    dx, dy = dy, dx
    
    package_clauses = []
    
    # Iterate over all the coordinates where p can fit
    for i in range(h - dy + 1):
        for j in range(w - dx + 1):
            
            patch_clauses = []
            # Iterate over the cells of p's patch
            for f1 in range(dy):
                for f2 in range(dx):
                    patch_clauses.append(B[i + f1][j + f2][p])
            
            package_clauses.append(And(patch_clauses))
    
    solver.add(Or(Not(R[p]), at_least_one(package_clauses)))

  0%|          | 0/8 [00:00<?, ?it/s]

In [10]:
%%time
solver.check()

CPU times: user 604 ms, sys: 1.74 ms, total: 606 ms
Wall time: 609 ms


sat

From Z3 model solution to file:

In [11]:
solution = np.zeros((h, w, n), dtype=bool)
model = solver.model()

for i in range(h):
    for j in range(w):
        for k in range(n):
            solution[i, j, k] = is_true(model[B[i][j][k]])

In [12]:
xy = {}
for p in range(n):
    y_ids, x_ids = solution[:, :, p].nonzero()
    #print(solution[:, :, p])
    x = np.min(x_ids)
    y = h-1-np.max(y_ids)
    xy[p] = [x, y]

In [13]:
xy

{0: [3, 9],
 1: [6, 0],
 2: [0, 7],
 3: [9, 6],
 4: [0, 0],
 5: [6, 4],
 6: [3, 0],
 7: [9, 0]}

In [14]:
output_filename = '../../pwp_utilities/17x17_sol.txt'
with open(output_filename, 'w') as f_out:
    f_out.write('{} {}\n'.format(w, h))
    f_out.write('{}\n'.format(n))
    for i in range(n):
        if is_true(model[R[i]]):
            f_out.write('{} {}\t{} {}\n'.format(DY[i], DX[i], xy[i][0], xy[i][1]))
            print('R')
        else:
            f_out.write('{} {}\t{} {}\n'.format(DX[i], DY[i], xy[i][0], xy[i][1]))

R
